In [1]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
from sclib import SoundcloudAPI, Track, Playlist
import json
import yt_dlp
import os
from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3
import re
import requests
from mutagen.id3 import ID3, APIC

#___________________________________________________________________________________

def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

#___________________________________________________________________________________

with open("key.json", "r") as file:
    api_tokens = json.load(file)

client_secret = api_tokens['client_secret']
client_id = api_tokens['client_id']
redirectURI = api_tokens['redirect']
username = api_tokens['username']

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirectURI,
                                               scope="playlist-read-private"))

with open("soundcloud_key.json", "r") as sound_file:
    sound_api_tokens = json.load(sound_file)

sound_id = sound_api_tokens['client_id']
sound_api = SoundcloudAPI(client_id=sound_id)

download_path = r'C:\Users\Zade\Downloads\Test'  # <------------------------- INSERT DOWNLOAD DIRECTORY HERE 

playlist_url = input("Enter the playlist URL (Spotify or SoundCloud): ")

#___________________________________________________________________________________

def download_from_youtube(query, download_path):
    sanitized_query = sanitize_filename(query)
    mp3_file_path = os.path.join(download_path, f"{sanitized_query}.mp3")
    thumbnail_file_path = os.path.join(download_path, f"{sanitized_query}.jpg")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '320',  # <------------------------- CHANGE SPOTIFY DOWNLOAD QUALITY (320kbps high quality, 128kbps average quality 
            },
            {
                'key': 'EmbedThumbnail',
            },
        ],
        'writethumbnail': True,
        'outtmpl': os.path.join(download_path, f"{sanitized_query}.%(ext)s"),
        'quiet': True
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            print(f"Downloading {query}...")
            ydl.download([f"ytsearch:{query}"])

            if os.path.exists(thumbnail_file_path):
                embed_thumbnail_in_mp3(mp3_file_path, thumbnail_file_path)
            else:
                print()
        except Exception as e:
            print()



#___________________________________________________________________________________



def add_metadata(file_path, artist, album): #spotify one
    if os.path.exists(file_path):
        audio = MP3(file_path, ID3=EasyID3)
        audio['artist'] = artist
        audio['album'] = album
        audio.save()

def add_metadata_sound(file_path, artist): #soundcloud one
    if os.path.exists(file_path):
        audio = MP3(file_path, ID3=EasyID3)
        audio['artist'] = artist
        audio.save()

def download_artwork(artwork_url, track_name): #soundcloud one
    if artwork_url:
        response = requests.get(artwork_url)
        if response.status_code == 200:
            artwork_path = os.path.join(download_path, f"{sanitize_filename(track_name)}_artwork.jpg")
            with open(artwork_path, 'wb') as image_file:
                image_file.write(response.content)

def embed_thumbnail_in_mp3(mp3_file_path, thumbnail_file_path): #spotify one
    if os.path.exists(mp3_file_path) and os.path.exists(thumbnail_file_path):
        audio = MP3(mp3_file_path, ID3=ID3)
        with open(thumbnail_file_path, 'rb') as img_file:
            audio.tags.add(
                APIC(
                    encoding=0,  # UTF-8
                    mime='image/jpeg',  # Ensure JPEG format for compatibility
                    type=3,  # Cover (front)
                    desc='Cover',
                    data=img_file.read()
                )
            )
        audio.save()
        print(f"Thumbnail embedded into {mp3_file_path}")
        os.remove(thumbnail_file_path)
    else:
        print(f"Could not find MP3 or thumbnail for embedding: {mp3_file_path}, {thumbnail_file_path}")

#___________________________________________________________________________________

def download_spotify_playlist(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    for item in tracks:
        track = item['track']
        track_name = sanitize_filename(f"{track['artists'][0]['name']} - {track['name']}")
        artist = track['artists'][0]['name']
        album = track['album']['name']
        
        download_from_youtube(track_name, download_path)
 
        file_path = os.path.join(download_path, f"{track_name}.mp3")
        add_metadata(file_path, artist, album)

#___________________________________________________________________________________

def download_soundcloud_playlist(url):
    playlist = sound_api.resolve(url)
    
    if isinstance(playlist, Playlist):
        for track in playlist.tracks:
            if isinstance(track, Track):
                track_name = sanitize_filename(f"{track.artist} - {track.title}.mp3")
                file_path = os.path.join(download_path, track_name)

                with open(file_path, 'wb+') as file:
                    track.write_mp3_to(file)

                if track.artwork_url:
                    response = requests.get(track.artwork_url)
                    if response.status_code == 200:
                        audio = MP3(file_path, ID3=ID3)
                        audio.tags.add(
                            APIC(
                                encoding=0,  # UTF-8, only use 0,1,2
                                mime='image/jpeg', 
                                type=3,  # Cover (front)
                                desc='Cover',
                                data=response.content
                            )
                        )
                        audio.save()

                add_metadata_sound(file_path, track.artist)
                print(f"Track downloaded: {track_name}")

#___________________________________________________________________________________

if "spotify.com" in playlist_url:
    playlist_id = playlist_url.split("/")[-1].split("?")[0]
    download_spotify_playlist(playlist_id)
elif "soundcloud.com" in playlist_url:
    download_soundcloud_playlist(playlist_url)
else:
    print("URL not recognised :((")

KeyboardInterrupt: Interrupted by user